In [1]:
import pycuda.autoinit
import pycuda.gpuarray as gpuarray
from pycuda.compiler import *
import pycuda.driver as cuda
import numpy as np
import numba

In [2]:
NUMEVENTS = 5000
AVENUMJETS = 100

numjets1 = np.random.poisson(AVENUMJETS, NUMEVENTS).astype(np.int32)
stops1 = np.cumsum(numjets1, dtype=np.int32)
starts1 = np.zeros_like(stops1)
starts1[1:] = stops1[:-1]

counts1 = stops1-starts1
offsets1 = np.zeros(len(numjets1)+1)
offsets1[1:] = stops1[:]

numjets2 = np.random.poisson(AVENUMJETS, NUMEVENTS).astype(np.int32)
stops2 = np.cumsum(numjets2, dtype=np.int32)
starts2 = np.zeros_like(stops2)
starts2[1:] = stops2[:-1]


counts2 = stops2-starts2
offsets2 = np.zeros(len(numjets2)+1)
offsets2[1:] = stops2[:]

In [3]:
@numba.jit()
def vectorized_search(offsets, content):
    index = np.arange(len(content), dtype=np.int32)                     
    below = np.zeros(len(content), dtype=np.int32)                      
    above = np.ones(len(content), dtype=np.int32) * (len(offsets) - 1)  
    while True:
        middle = (below + above) // 2

        change_below = offsets[middle + 1] <= index                  
        change_above = offsets[middle] > index                        

        if not np.bitwise_or(change_below, change_above).any():    
            break
        else:
            below = np.where(change_below, middle + 1, below)      
            above = np.where(change_above, middle - 1, above)      

    return middle

In [4]:
pairs_indices = np.zeros(NUMEVENTS+1, dtype=np.int32)
pairs_indices[1:] = np.cumsum(counts1*counts2, dtype=np.int32)
pairs_indices = pairs_indices

In [5]:
pairs_contents = np.arange(pairs_indices[-1]).astype(np.int32)
pairs_parents = vectorized_search(pairs_indices, pairs_contents)
pairs_parents = pairs_parents.astype(np.int32)

In [6]:
mod = SourceModule('''
__global__ void combinations(int* starts1,int* starts2,int* counts2,int* pairs_parents,int* pairs_indices,int* left,int* right,int* numpairs)
{
    unsigned int idx = threadIdx.x + blockIdx.x*blockDim.x;
    int temp;
    int tid = threadIdx.x;
    // __shared__ int spairs_parents[512], spairs_indices[512], scounts2[512], sstarts1[512], sstarts2[512],sleft[512], sright[512];
    
    if (idx >= numpairs[0])
    {
        return;
    }
    /*
    spairs_parents[tid] = pairs_parents[idx];
    spairs_indices[tid] = pairs_indices[idx];
    scounts2[tid] = counts2[idx];
    sstarts1[tid] = starts1[idx];
    sstarts2[tid] = starts2[idx];
    __syncthreads();
    
    if (scounts2[spairs_parents[tid]]>0)
        {
            temp[0] = (idx-spairs_indices[spairs_parents[tid]])/scounts2[pairs_parents[tid]];
            sleft[tid] = sstarts1[spairs_parents[tid]] + temp[0];
            sright[tid] = sstarts2[spairs_parents[tid]] + (idx-spairs_indices[spairs_parents[tid]])-scounts2[spairs_parents[tid]]*temp[0];
        }
        __syncthreads();
        
        left[idx] = sleft[tid];
        right[idx] = sright[tid];
        __syncthreads();
    */
    
    if (counts2[pairs_parents[idx]]>0)
        {
            temp = (idx-pairs_indices[pairs_parents[idx]])/counts2[pairs_parents[idx]];
            left[idx] = starts1[pairs_parents[idx]] + temp;
            right[idx] = starts2[pairs_parents[idx]] + (idx-pairs_indices[pairs_parents[idx]])-counts2[pairs_parents[idx]]*temp;
        }
        __syncthreads();
}
''')

In [7]:
func = mod.get_function('combinations')

In [8]:
gpu_starts1 = gpuarray.to_gpu(starts1)
gpu_starts2 = gpuarray.to_gpu(starts2)
gpu_counts2 = gpuarray.to_gpu(counts2)
gpu_pairs_parents = gpuarray.to_gpu(pairs_parents)
gpu_pairs_indices = gpuarray.to_gpu(pairs_indices)
left = gpuarray.zeros(pairs_indices[-1], dtype=np.int32)-1
right = gpuarray.zeros_like(left)-1
numpairs = gpuarray.to_gpu(np.array([pairs_indices[-1]]).astype(np.int32))
numthreads = 512
numblocks = int(np.ceil(pairs_indices[-1]/numthreads))

In [9]:
start = cuda.Event()
stop = cuda.Event()
start.record()
func(gpu_starts1,gpu_starts2,gpu_counts2,gpu_pairs_parents,gpu_pairs_indices,left,right,numpairs, block=(numthreads,1,1), grid = (numblocks,1))

stop.record()
stop.synchronize()
print ("Total time taken = {} milliseconds".format(start.time_till(stop)))

Total time taken = 44.93766403198242 milliseconds


In [10]:
# Uncomment if need to view output
#for i in range(6):
#   print("Event {}\n Left {}\nRight {}\n\n".format(i, left[pairs_indices[i]:pairs_indices[i+1]], right[pairs_indices[i]:pairs_indices[i+1]]))

In [11]:
@numba.jit()
def product_cpu(starts1,starts2,counts,pairs_parents,pairs_indices,left, right):
    pairs_contents = np.arange(pairs_indices[-1]).astype(np.int)
    left[pairs_contents] = starts1[pairs_parents[pairs_contents]] + np.floor((pairs_contents-pairs_indices[pairs_parents[pairs_contents]])/counts2[pairs_parents[pairs_contents]]).astype(np.int)
    right[pairs_contents] = starts2[pairs_parents[pairs_contents]]+(pairs_contents-pairs_indices[pairs_parents[pairs_contents]])-counts2[pairs_parents[pairs_contents]]*np.floor((pairs_contents-pairs_indices[pairs_parents[pairs_contents]])/counts2[pairs_parents[pairs_contents]])

In [12]:
cleft = np.empty(pairs_indices[-1])
cright = np.empty_like(cleft)

In [14]:
#CPU timing
%%timeit
product_cpu(starts1,starts2,counts2,pairs_parents,pairs_indices,cleft,cright)

7.56 s ± 163 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
